In [1]:
# 실제 AI한테 더하기 계산식 (더하기 밖에 학습을 안시켜서 더하기밖에 못함) 입력해보기

##1. 계산식 입력
##2. 계산식 문장을 전처리
##3. 계산식 문장을 encoder와 decoder에 넣어서 y를 출력
##4. y값 분석해서 다시 문장으로 바꾸기


In [2]:
import os
import sys
sys.path.append(os.path.join(os.path.dirname(""), ".."))

import custom
import numpy

import torch
import torch.nn as nn

In [3]:
# 신경망 커스텀 class 선언

# 함수들 만들기
class Encoder(nn.Module) :
    def __init__(self, embedding_vector) :
        super().__init__()
        self.embedding = nn.Embedding.from_pretrained(embedding_vector, freeze = True, padding_idx = 0) # 라벨값을 벡터로 바꾸는 Embedding 함수
        self.rnn = nn.LSTM(embedding_vector.shape[1], embedding_vector.shape[1], batch_first = True, bidirectional = True) #임베딩 열 크기를 받아서 임베딩 열 크기를 출력
    def forward(self, x) :
        x = self.embedding(x)
        y, hc = self.rnn(x)
        return y, hc

class Decoder(nn.Module) :
    def __init__(self, embedding_vector) :
        super().__init__()
        self.embedding = nn.Embedding.from_pretrained(embedding_vector, freeze = True, padding_idx = 0)
        self.rnn = nn.LSTM(embedding_vector.shape[1] * 3, embedding_vector.shape[1], batch_first = True, bidirectional = True)
        self.f = nn.Linear(embedding_vector.shape[1] * 4, embedding_vector.shape[0]) #RNN을 계산하고 난 뒤 벡터값을 다시 라벨값으로 바꾸는 용도, 단어 출력 용도
        self.encoder_h_context = None #encoder_h 가공값 저장할 변수
    def forward(self, encoder_output, encoder_hc, t = None) : #문장 여러개 처리하기 위해서 encoder_output.shape가 필요함
        #decoder 학습 방법 2가지 (greedy, teaching_force)
        #encoder_h 가공
        encoder_h_forward = encoder_hc[0][0:1,:,:]
        encoder_h_backward = encoder_hc[0][1:2,:,:] #encoder의 h값 반으로 쪼개기
        self.encoder_h_context = torch.concat([encoder_h_forward, encoder_h_backward], dim = -1).transpose(0,1)
        
        batch_size = encoder_output.shape[0] #문장 갯수 정하기 = encoder에서 처리한 문장 갯수
        decoder_input = torch.zeros(batch_size, 1).type(torch.long).to(encoder_output.device) #처음 넣는 단어 (0번째 단어인 padding)
        decoder_hc = encoder_hc
        decoder_output_list = [] #단어들 모아서 문장으로 만들어 출력

        for i in range(4) : #t의 문자갯수만큼 반복
            decoder_output, decoder_hc = self.forward_cal(decoder_input, decoder_hc)
            decoder_output_list.append(decoder_output)

            if t is None : #greedy (t가 없음)
                decoder_input = decoder_output.argmax(dim = -1).detach()
            else : #teaching-force (t가 있음)
                decoder_input = t[:, i:i+1]

        decoder_output_list = torch.cat(decoder_output_list, dim = 1) #for 문 종료 후 list로 모아놓은 tensor들을 tensor로 합치는 작업
        return decoder_output_list, decoder_hc
        
    def forward_cal(self, x, h) : #실제 신경망 계산 함수
        x = self.embedding(x) #라벨을 벡터로
        x = torch.concat([self.encoder_h_context, x], dim = -1)
        output, h = self.rnn(x, h) #처음 h값을 지정하고 싶으면 x 뒤에 h값 입력하세요
        output = torch.concat([self.encoder_h_context, output], dim = -1)
        output = self.f(output) #벡터 계산값을 다시 라벨값으로 변환
        return output, h


In [4]:
# 신경망 불러오기
encoder = torch.load("num_encoder.pt", weights_only=False).to("cpu") #신경망 불러와서 cpu로 변환
decoder = torch.load("num_decoder.pt", weights_only=False).to("cpu")

encoder.eval() #dropout 꺼주기
decoder.eval()

print(encoder)
print(decoder)

Encoder(
  (embedding): Embedding(12, 11, padding_idx=0)
  (rnn): LSTM(11, 11, batch_first=True, bidirectional=True)
)
Decoder(
  (embedding): Embedding(12, 11, padding_idx=0)
  (rnn): LSTM(33, 11, batch_first=True, bidirectional=True)
  (f): Linear(in_features=44, out_features=12, bias=True)
)


In [5]:
## 숫자-라벨링 사전 정의

num_dict = {' ': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '0': 10, '+': 11}

print(num_dict)

{' ': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '0': 10, '+': 11}


In [6]:

##1. 계산식 입력
query = input("계산식 입력하세요 (7개 문자까지만 가능) : ")

##2. 계산식 문장을 전처리
query = list(query) #문자 하나단위로 끊기
vector = custom.word_vectorize(query, num_dict, 7, False, " ", " ") #문자 라벨링
vector = [vector] #2차원으로 만들기
tensor = torch.tensor(vector, dtype = torch.long) #tensor로 변환

##3. 계산식 문장을 encoder와 decoder에 넣어서 y를 출력
encoder_y, encoder_hc = encoder(tensor)
y, _ = decoder(encoder_y, encoder_hc)
print(y)
print(y.shape)

##4. y값 분석해서 다시 문장으로 바꾸기
print("결과값 : ", end = "")
for a in y[0] :
    print(list(num_dict.keys())[a.argmax(dim = -1)] ,end = "")


계산식 입력하세요 (7개 문자까지만 가능) :  5+12


tensor([[[-18.9582,  28.4378,  24.1172,  13.4422,  -3.2165, -13.4384, -11.5027,
          -14.3919, -25.1196, -22.7492,  26.5713,  -9.1549],
         [  7.4898,  -0.7630,  -9.4761, -13.0095,  -9.7613,  -1.0292,  10.0346,
           13.9712,   9.6204,   0.5125,   6.4941, -10.0765],
         [ 62.3868,  -9.5715, -17.5198, -21.5635, -14.0712,  -4.3158,   8.9685,
           16.4402,  18.6293,   7.4377,  -5.3554,  -7.8343],
         [ 45.4433,   4.7260,  15.9797,  17.1387,  11.1716,   4.3421,  -3.8997,
          -17.5642, -32.0090, -31.1792,  -7.4532,  -8.9791]]],
       grad_fn=<CatBackward0>)
torch.Size([1, 4, 12])
결과값 : 17  